# GEM radix Lagrangean decomposition

In [1]:
REG_WEIGHT   = 0. #1e-4
MAX_NONZERO  = None #48*2
PREVENT_ZERO = True

In [2]:
%load_ext line_profiler

In [3]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from cobra.io import load_json_model
from six import iteritems
import numpy as np
import cobra

In [4]:
#ijomc = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')
ijomc = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc.json')

In [5]:
print(ijomc.optimize())
mu_crowd0 = ijomc.reactions.BIOMASS_Ec_iJO1366_core_53p95M
print(mu_crowd0)

<Solution 0.97 at 0x7fcb852d6fd0>
BIOMASS_Ec_iJO1366_core_53p95M


In [6]:
df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')

ex_rxns = [r for r in df_meas.ex_rxn.unique() if ijomc.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()

#N_CONDS = len(conds)
N_CONDS = 2

df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])

if N_CONDS==1:
    df_conds = df_conds[ df_conds.cond=='acetate']
elif N_CONDS<=3:
    df_conds = df_conds[ df_conds.cond.isin(['glucose','acetate','succinate'][0:N_CONDS])]
else:
    df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

df_conds.loc[ (df_conds.cond=='acetate') & (df_conds.rxn=='EX_ac_e'), 'lb'] = -20

# Make radix problem

In [7]:
from dynamicme.estimate import RadixEstimator

In [8]:
df_Y = df_meas.rename(columns={'growth_rate_1_h':'output', 'substrate':'cond'})
df_Y.loc[:,'output_id'] = 'BIOMASS_Ec_iJO1366_core_53p95M'
df_X = df_conds

In [9]:
from dynamicme.decomposition import LagrangeMaster, LagrangeSubmodel

In [10]:
conds = df_conds.cond.unique()
sub_dict = {}
for cond in conds:
    df_Xk = df_X[ df_X.cond==cond]
    df_Yk = df_Y[ df_Y.cond==cond]
    estk = RadixEstimator()
    estk.fit(ijomc, df_Xk, df_Yk, reg_weight = REG_WEIGHT, max_nonzero_binaries=MAX_NONZERO, optimize=False)    
    sub = LagrangeSubmodel(estk.stacker.model, cond)
    sub_dict[cond] = sub

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1


In [11]:
master = LagrangeMaster(estk.stacker.model)
master.add_submodels(sub_dict)

### Two-phase

In [12]:
master.time_limit = 2*3600
#master.max_iter = 500
master.verbosity = 1
master.print_iter = 5
master.gaptol = 0.01
master.solve_relaxed()
sol_master = master.optimize()

    Iter                    UB                    LB       gap relgap(%)     delta                       Time(s)
  ------   -------------------   -------------------  -------- ---------  -------- -----------------------------
               Dual       Feas        Sub       Best                                   total    master       sub
       0       1000       1000 -8.574e-08 -8.574e-08      1000       100       0.5  6.242681  0.034142  3.124950
       5     0.4423    0.08027     -40.34 -8.574e-08   0.08027       100    0.0156  88.72628  0.075902  15.66285
      10      4.454    0.08027      -1173 -8.574e-08   0.08027       100  0.000488  188.7139  0.088902  4.554474
      15      34.48    0.08027 -2.269e+04 -8.574e-08   0.08027       100  1.53e-05  246.5742  0.134131  4.023077
      20      61.98    0.08027  -1.81e+05 -8.574e-08   0.08027       100  4.77e-07  305.5671  0.149304  5.749022
      25      47.34    0.08027  -5.29e+05 -8.574e-08   0.08027       100  1.49e-08  371.9440  0.

/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition.py:2097 UserWarning: Solution available but Master solver status=time_limit (9).


Stopping due to time limit of 7200 seconds.
    Iter                    UB                    LB       gap relgap(%)     delta                       Time(s)
  ------   -------------------   -------------------  -------- ---------  -------- -----------------------------
               Dual       Feas        Sub       Best                                   total    master       sub
       0     -989.9     -989.9        nan    -1e+100    1e+100  1.01e+99       0.5  8.911261  1.669152  3.538034
       1   0.002504     -989.9     -0.871     -0.871      -989    -99.91       0.5  1795.668  12.91090  1771.626
relgap (-0.99912) <= gaptol (0.01). Finished.


In [13]:
var_cons_dict = estk.var_cons_dict
powers = estk.powers
digits = estk.digits
radix  = estk.radix

In [14]:
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol_master[yid]
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

kfit_changed = [(k,v, abs(v-a0)/a0) for k,v in iteritems(kfit_dict) if abs(v-a0)/a0>1e-6]
print('Changed keffs: %d/%d' % (len(kfit_changed), len(var_cons_dict)))
#----------------------------------------
# Starting from basal model
perrs = []
csrcs = df_conds.cond.unique()
for csrc in csrcs:        
    ijofit = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc.json')
    crowding = ijofit.metabolites.get_by_id('crowding')
    df_condi = df_conds[ df_conds.cond==csrc]    
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.lower_bound = row['lb']
        rxn.upper_bound = row['ub']

    for rid,kfit in iteritems(kfit_dict):
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.add_metabolites({crowding:kfit}, combine=False)
    
    ijofit.optimize()
    
    mu_measi = df_meas[ df_meas.substrate==csrc].growth_rate_1_h.iloc[0]
    mu_fiti = ijofit.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x
    
    # Get unfit
    for rxn in ijofit.metabolites.crowding.reactions:
        rxn._metabolites[crowding] = a0
    ijofit.optimize()
    mu_unfiti = ijofit.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x
    err0= 100*(mu_unfiti-mu_measi)/mu_measi
    err = 100*(mu_fiti - mu_measi)/mu_measi
    derr= 100*(abs(err)-abs(err0))/abs(err0)
    perrs.append(err)
    print('Cond=%s. mu_meas=%g. mu_sim=%g (unfit=%g, error=%.3g%%). Error=%.3g%% (%.3g%% change)' % (
        csrc, mu_measi, mu_fiti, mu_unfiti, err0, err, derr))
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)        
        print('\t%s uptake=%g' % (rxn.id, rxn.x))
perrs = np.array(perrs)
print("Absolute percent error: %g%% -- %g%%"%(min(abs(perrs)), max(abs(perrs))))
print("Mean abs percent error = %g%%"%(np.mean(abs(perrs))))
print("Median abs percent error = %g%%"%(np.median(abs(perrs))))

Changed keffs: 2018/2018
Cond=glucose. mu_meas=0.74. mu_sim=0.872161 (unfit=0.982372, error=32.8%). Error=17.9% (-45.5% change)
	EX_glc__D_e uptake=-10
	EX_malt_e uptake=0
	EX_man_e uptake=0
	EX_fru_e uptake=0
	EX_gal_e uptake=0
	EX_succ_e uptake=0
	EX_mal__L_e uptake=0
	EX_glyc_e uptake=0
	EX_lac__L_e uptake=0
	EX_ac_e uptake=0
Cond=acetate. mu_meas=0.256. mu_sim=0.507518 (unfit=0.507518, error=98.2%). Error=98.2% (1.94e-10% change)
	EX_glc__D_e uptake=0
	EX_malt_e uptake=0
	EX_man_e uptake=0
	EX_fru_e uptake=0
	EX_gal_e uptake=0
	EX_succ_e uptake=0
	EX_mal__L_e uptake=0
	EX_glyc_e uptake=0
	EX_lac__L_e uptake=0
	EX_ac_e uptake=-20
Absolute percent error: 17.8596% -- 98.2491%
Mean abs percent error = 58.0544%
Median abs percent error = 58.0544%


## Total error

In [17]:
y_sames = []
for y in sub._ys:
    sub1 = sub_dict['acetate']
    sub2 = sub_dict['glucose']
    y1 = sub1.x_dict[y.VarName]
    y2 = sub2.x_dict[y.VarName]
    y_sames.append(abs(y1-y2)<1e-10)
    print('%20.18s%12.10s%12.10s%12.10s'%(y.VarName,y1,y2, abs(y1-y2)<1e-10))    

       binary_CSND00         0.0         0.0        True
       binary_CSND01         1.0         1.0        True
       binary_CSND02         1.0         1.0        True
    binary_GTPDPDP00         0.0         0.0        True
    binary_GTPDPDP01         0.0         0.0        True
    binary_GTPDPDP02         1.0         1.0        True
  binary_ASNt2rpp_ab         0.0         0.0        True
  binary_ASNt2rpp_ab         0.0         0.0        True
  binary_ASNt2rpp_ab         1.0         1.0        True
      binary_NTD1000         0.0         0.0        True
      binary_NTD1001         1.0         1.0        True
      binary_NTD1002         0.0         0.0        True
     binary_TDPGDH00         0.0         0.0        True
     binary_TDPGDH01         1.0         1.0        True
     binary_TDPGDH02         1.0         1.0        True
      binary_NTD1200         0.0         0.0        True
      binary_NTD1201         1.0         1.0        True
      binary_NTD1202         1.

In [16]:
print('num same = %s/%s'%(sum(y_sames), len(y_sames)))

num same = 6054/6054
